Data Preprocessing

In [15]:
# Loading Whole Dataset
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import KFold
import torch

# Define transformations (resize, normalize, etc.)
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224
    transforms.ToTensor(),          # Convert images to PyTorch tensors
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize
])

# Load the dataset (class labels are inferred from folder names)
full_dataset = datasets.ImageFolder('./403-Project3-Dataset/Dataset', transform=transform)


In [16]:
print(f'Total number of images: {len(full_dataset)}')
print(f'Class names: {full_dataset.classes}')

from collections import Counter

# Get all the labels (targets)
targets = [label for _, label in full_dataset]

# Count occurrences of each class
class_counts = Counter(targets)

# Map class indices to class names
class_names = full_dataset.classes
class_distribution = {class_names[idx]: count for idx, count in class_counts.items()}

print(f'Class distribution: {class_distribution}')

Total number of images: 485
Class names: ['Alex', 'Kelly']
Class distribution: {'Alex': 256, 'Kelly': 229}


Creating K Folds

In [17]:
from sklearn.model_selection import KFold

k_folds = 5
kfold = KFold(n_splits=k_folds, shuffle=True, random_state=42)

Model 1: CNN

In [18]:
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        
        # CONVL 1
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(16) 
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.dropout1 = nn.Dropout(0.25)  

        # CONVL 2
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(32)  
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.dropout2 = nn.Dropout(0.25)  
        
        # CONVL 3
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(64)  
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.dropout3 = nn.Dropout(0.25)  

        # MLP Layers
        self.fc1 = nn.Linear(256 * 14 * 14, 128)  # Adjust size according to input dimensions
        self.dropout_fc1 = nn.Dropout(0.5)  # Dropout in FC Layer
        self.fc2 = nn.Linear(128, 2)  # 2 classes for output

    def forward(self, x):
        x = self.pool1(F.relu(self.bn1(self.conv1(x))))
        x = self.dropout1(x)
        
        x = self.pool2(F.relu(self.bn2(self.conv2(x))))
        x = self.dropout2(x)
        
        x = self.pool3(F.relu(self.bn3(self.conv3(x))))
        x = self.dropout3(x)
        
        x = x.view(-1, 256 * 14 * 14)  # Flatten for fully connected layer
        x = F.relu(self.fc1(x))
        x = self.dropout_fc1(x)
        x = self.fc2(x)
        return x


CNN Training

In [ ]:



for fold, (train_idx, val_idx) in enumerate(kfold.split(full_dataset)):
    print(f'Fold {fold + 1}')
    
    # Create data loaders for train and validation sets
    train_subset = Subset(full_dataset, train_idx)
    val_subset = Subset(full_dataset, val_idx)
    train_loader = DataLoader(train_subset, batch_size=64, shuffle=True)
    val_loader = DataLoader(val_subset, batch_size=64, shuffle=False)
    
    # Initialize the model, optimizer, and learning rate scheduler
    model = CNNModel()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()
    
    # Training loop
    model.train()
    for epoch in range(5):  # Number of epochs per fold
        running_loss = 0.0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
    
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        
        train_accuracy = 100 * correct / total
        print(f'Epoch {epoch + 1} | Loss: {running_loss / len(train_loader):.4f} | Training Accuracy: {train_accuracy:.2f}%')
    
    # Validation loop
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    print(f'Validation Accuracy for Fold {fold + 1}: {100 * correct / total:.2f}%')

Fold 1
Epoch 1, Loss: 13.48855994428907
Epoch 2, Loss: 6.4497054644993375
Epoch 3, Loss: 2.495854616165161
Epoch 4, Loss: 2.0422814914158414
Epoch 5, Loss: 1.0848817144121443
Validation Accuracy for Fold 1: 68.04%
Fold 2
Epoch 1, Loss: 7.043987887246268
Epoch 2, Loss: 4.280316097395761
Epoch 3, Loss: 1.4477314502000809
Epoch 4, Loss: 1.0156748976026262
Epoch 5, Loss: 0.6173615796225411
Validation Accuracy for Fold 2: 68.04%
Fold 3
Epoch 1, Loss: 16.547843422208512
Epoch 2, Loss: 1.5998685785702296
Epoch 3, Loss: 0.6823646596499852
Epoch 4, Loss: 0.6831548299108233
Epoch 5, Loss: 0.7262020196233477
Validation Accuracy for Fold 3: 48.45%
Fold 4
Epoch 1, Loss: 16.86480869565691
Epoch 2, Loss: 5.416777150971549
Epoch 3, Loss: 2.713440581623997
Epoch 4, Loss: 1.7170007739748274
Epoch 5, Loss: 0.7727605317320142
Validation Accuracy for Fold 4: 61.86%
Fold 5
Epoch 1, Loss: 10.086397920336042
Epoch 2, Loss: 1.8452890472752708
Epoch 3, Loss: 0.9130093966211591
Epoch 4, Loss: 0.603099490915026
E